# Training the lenet model

In [ ]:
import keras
import tensorflow as tf
import tf2onnx
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

## Handles input data

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Design the model

In [ ]:
model = Sequential()

### Layer 1
Conv layer 1

In [ ]:
model.add(Conv2D(filters = 6, 
                 kernel_size = 5, 
                 strides = 1, 
                 activation = 'relu', 
                 input_shape = (28,28,1)))

Pooling Layer 1

In [ ]:
model.add(MaxPooling2D(pool_size = 2, strides = 2))

### Layer 2

Conv layer 2

In [ ]:
model.add(Conv2D(filters = 16, 
                 kernel_size = 5,
                 strides = 1,
                 activation = 'relu',
                 input_shape = (14,14,6)))

Pooling layer 2

In [ ]:
model.add(MaxPooling2D(pool_size = 2, strides = 2))

Flatten

In [ ]:
model.add(Flatten())

### Layer 3
Fully Connected Layer 3

In [ ]:
model.add(Dense(units = 120, activation = 'relu'))

### Layer 4
Fully Connected Layer 4

In [ ]:
model.add(Dense(units = 84, activation = 'relu'))

### Layer 5
Output layer

In [ ]:
model.add(Dense(units = 10, activation = 'softmax'))
model.summary()

### # define lossy optimization function

In [ ]:
filepath = "mnist-lenet-weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1,
                                 save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.compile(optimizer='adam', loss='mean_squared_error',
        metrics=['accuracy'])

### Perform the training

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=64,
              callbacks=callbacks_list)

### Convert model from keras (h5) to ONNX (onnx)

In [ ]:
tf2onnx.convert.from_keras(model, inputs_as_nchw=[model.inputs[0].name],
        output_path = "lenet.onnx")